In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold, ShuffleSplit
from sklearn.model_selection import RepeatedStratifiedKFold
SEED=2020
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from scipy import stats
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import eli5 
from eli5.sklearn import PermutationImportance
import shap #for SHAP values
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, SelectKBest, RFE, chi2
# from xgboost.sklearn import XGBClassifier 
import xgboost as xgb
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from xgboost import plot_tree

In [ ]:
# load dataset and drop duplicates
df = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
df = df.drop_duplicates()

In [ ]:
# take a look for dataset
df.sample(20)

In [ ]:
sns.set(style = "white")
g = sns.PairGrid(df.loc[:,["age","trestbps","chol", 'thalach', 'oldpeak']],diag_sharey = False,)
g.map_lower(sns.kdeplot,cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot,lw =3)
plt.show()

In [ ]:
plt.figure(figsize = (10, 12))

# iterate 
for i, source in enumerate(["age","trestbps","chol", 'thalach', 'oldpeak']):
    
    # create a new subplot for each source
    plt.subplot(3, 2, i + 1)
    # plot repaid loans
    sns.kdeplot(df.loc[df['target'] == 0, source], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(df.loc[df['target'] == 1, source], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)
    

In [ ]:
sns.countplot(x="target", data=df)
df.loc[:,'target'].value_counts()

In [ ]:
## basic algo accuracy:
print('Basic algo accuracy')
round(df['target'].value_counts(normalize=True).max(), 2)

In [ ]:
## correlation table
pd.DataFrame(abs(df.corr()['target']).sort_values(ascending = False))

In [ ]:
# drop outliers with z-score over 99.9%
z = np.abs(stats.zscore(df._get_numeric_data()))
df= df[(z < 3).all(axis=1)]
print('Outliers dropped')

# df = df[(df['chol'] <= 326.9) & (df['oldpeak'] <=3.4)].reset_index(drop=True)

In [ ]:
# some new features based on continious features
df['ageCat'] = pd.cut(df['age'].astype(int), 8)
df['trestbpsCat'] = pd.cut(df['trestbps'], 8)
df['cholCat'] = pd.cut(df['chol'], 8)
df['thalachCat'] = pd.cut(df['thalach'], 8)
df['oldpeakCat'] = pd.cut(df['oldpeak'], 8)

label = LabelEncoder()
df['ageCat'] = label.fit_transform(df['ageCat'])
df['trestbpsCat'] = label.fit_transform(df['trestbpsCat'])
df['cholCat'] = label.fit_transform(df['cholCat'])
df['thalachCat'] = label.fit_transform(df['thalachCat'])
df['oldpeakCat'] = label.fit_transform(df['oldpeakCat'])

df['trestbpsLog'] = np.log1p(df['trestbps'])
df['cholLog'] = np.log1p(df['chol'])



In [ ]:
## more detailed info as for dataset
df.describe().T

In [ ]:
df.sample(20)

In [ ]:
for x in df.drop(['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'trestbpsLog', 'cholLog', 'target'], axis=1):
    print('Target Correlation by:', x)
    print(df[[x, 'target']].groupby(x, as_index=False).mean())
    print('-'*10, '\n')

In [ ]:
## distribution and re-checking for outliers in numeric features
import seaborn as sns
import matplotlib.pyplot as plt
features = df.drop('target', axis=1).columns
for i in features:
    sns.boxplot(x="target",y=i,data=df)
    plt.title(i+" by "+"target")
    plt.show()

In [ ]:
df.columns

In [ ]:
## feature creation script, thanks to @ Vitalii Mokin

def feature_creation(df):
    for i in ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'ageCat',
       'trestbpsCat', 'cholCat', 'thalachCat', 'oldpeakCat', 'trestbpsLog',
       'cholLog']:
        for j in ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal','ageCat',
       'trestbpsCat', 'cholCat', 'thalachCat', 'oldpeakCat', 'trestbpsLog',
       'cholLog']:
            df[i + "*" + j] = (df[i]*df[j]) # based on multiplication
            df[i + "_" + j] = df[i].astype('str') + "_" + df[j].astype('str') ## and concatenation

    return df

df = feature_creation(df)

In [ ]:
# # checking for Object columns & Encoding new categorical features

lencoders = {}
for col in df.select_dtypes(include=['object']).columns:
    lencoders[col] = LabelEncoder()
    df[col] = lencoders[col].fit_transform(df[col])

In [ ]:
## let's reduce memory for clear mind

def reduce_memory_usage(df):
    """ The function will reduce memory of dataframe
    Note: Apply this function after removing missing value"""
    intial_memory = df.memory_usage().sum()/1024**2
    print('Intial memory usage:',intial_memory,'MB')
    for col in df.columns:
        mn = df[col].min()
        mx = df[col].max()
        if df[col].dtype != object:            
            if df[col].dtype == int:
                if mn >=0:
                    if mx < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)
                    elif mx < np.iinfo(np.uint64).max:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
            if df[col].dtype == float:
                df[col] =df[col].astype(np.float32)
    
    red_memory = df.memory_usage().sum()/1024**2
    print('Memory usage after complition: ',red_memory,'MB')
    
reduce_memory_usage(df)

In [ ]:
df.info()

In [ ]:
# Threshold for removing correlated variables, thanks to @ Vitalii Mokin
threshold = 0.9  ## optimal level 0.9
# Absolute value correlation matrix
corr_matrix = df.corr().abs().round(2)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Select columns with Pearson's correlations above threshold
collinear_features = [column for column in upper.columns if any(upper[column] > threshold)]
features_filtered = df.drop(columns = collinear_features)
print('The number of features that passed the collinearity threshold: ', features_filtered.shape[1])
features_best = []
features_best.append(features_filtered.columns.tolist())

In [ ]:
# def corr_df(x, corr_val):
#     '''
#     Obj: Drops features that are strongly correlated to other features.
#           This lowers model complexity, and aids in generalizing the model.
#     Inputs:
#           df: features df (x)
#           corr_val: Columns are dropped relative to the corr_val input (e.g. 0.8)
#     Output: df that only includes uncorrelated features
#     '''

#     # Creates Correlation Matrix and Instantiates
#     corr_matrix = x.corr()
#     iters = range(len(corr_matrix.columns) - 1)
#     drop_cols = []

#     # Iterates through Correlation Matrix Table to find correlated columns
#     for i in iters:
#         for j in range(i):
#             item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
#             col = item.columns
#             row = item.index
#             val = item.values
#             if abs(val) >= corr_val:
#                 # Prints the correlated feature set and the corr val
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
#                 drop_cols.append(i)

#     drops = sorted(set(drop_cols))[::-1]

#     # Drops the correlated columns
#     for i in drops:
#         col = x.iloc[:, (i+1):(i+2)].columns.values
#         x = x.drop(col, axis=1)
#     return x
# df = corr_df(df, 0.92)

In [ ]:
# updated dataframe with filtred features
df=df[features_best[0]]

In [ ]:
# take a look for dataframe
def basic_details(df):
    b = pd.DataFrame()
    b['Missing value, %'] = round(df.isnull().sum()/df.shape[0]*100)
    b['N unique value'] = df.nunique()
    b['dtype'] = df.dtypes
    return b
basic_details(df)

In [ ]:
# more detailed
df.describe(include='all').T

In [ ]:
# some visualisations of features
df_hist = df.select_dtypes(exclude = ['bool','object'])
df_hist.hist(figsize = [15,15],bins = 50)
plt.show()

In [ ]:
# define cross-val strategy
cv_split = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=SEED)
#split dataset
X = df.drop('target', axis=1)
y = df.target
# Add zeros per row as extra feature
X['n0'] = (X == 0).sum(axis=1)
# add more features
# poly = PolynomialFeatures(degree=3)
# X=poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)


## scale features
# scaler = preprocessing.RobustScaler()
# scaler = preprocessing.StandardScaler() 
scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.Normalizer()
# scaler = preprocessing.QuantileTransformer()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)
X = pd.DataFrame(scaler.transform(X), columns = X.columns)

# alternative scale strategy
# X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))
# X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test))
# X=(X - np.min(X)) / (np.max(X) - np.min(X))

# standartization strategy as alternative?
# X = (X - X.mean()) / (X.std())
# X_train = (X_train - X_train.mean()) / (X_train.std())
# X_test = (X_test - X_test.mean()) / (X_test.std())

In [ ]:
rf = RandomForestClassifier(max_depth=3, n_estimators=50, random_state=SEED)
# rf.fit(X_train, y_train)
# acc_rf_train = round(rf.score(X_train, y_train)*100,2) 
# acc_rf_test = round(rf.score(X_test,y_test)*100,2)
scores = cross_val_score(rf, X, y, cv=cv_split, scoring = 'accuracy')
print("Random Forest based on full dataset")
# print("Training Accuracy: % {}".format(acc_rf_train))
# print("Testing Accuracy: % {}".format(acc_rf_test))
print("RF CV Accuracy Score Mean:", scores.mean())
# print("Standard Deviation:", scores.std())

In [ ]:
lr=LogisticRegression(C=0.3, random_state=SEED, max_iter=100000)
# lr.fit(X_train,y_train)
scores = cross_val_score(lr, X, y, cv=cv_split, scoring = 'accuracy')
# acc_log_train = round(lr.score(X_train, y_train)*100,2) 
# acc_log_test = round(lr.score(X_test,y_test)*100,2)
print("Logistic Regression based on full dataset")
# print("Training Accuracy: % {}".format(acc_log_train))
# print("Testing Accuracy: % {}".format(acc_log_test))
print("LR CV Accuracy Score Mean:", scores.mean())
# print("Standard Deviation:", scores.std())

In [ ]:
## Feature ranking with recursive feature elimination and cross-validated selection of the best number of features
# classifier = RandomForestClassifier(random_state=SEED)
# classifier = ExtraTreesClassifier(random_state=SEED)
classifier = LogisticRegression(random_state=SEED)
# classifier = DecisionTreeClassifier()
selector = RFECV(classifier, step = 1, cv=cv_split, n_jobs=-1,verbose=1,  scoring='accuracy')
selector.fit(X, y)
print('The optimal number of features is {}'.format(selector.n_features_))
features_rfecv = [f for f,s in zip(X, selector.support_) if s]
print('The selected features are:')
print ('{}'.format(features_rfecv)) ## optimal features list

In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (roc auc)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.savefig('feature_auc_nselected.png', bbox_inches='tight', pad_inches=1)

In [ ]:
## let's compare RF after feature selection
rf = RandomForestClassifier(max_depth=2, n_estimators=25, random_state=SEED)
X_rfe = X[features_rfecv]
rf.fit(X_rfe, y)
# acc_log_train = round(rf.score(X_train_rfe, y_train)*100,2) 
# acc_log_test = round(rf.score(X_test[features_rfecv],y_test)*100,2)
scores = cross_val_score(rf, X_rfe, y, cv=cv_split, scoring = 'accuracy')
# print("Training Accuracy: % {}".format(acc_log_train))
# print("Testing Accuracy: % {}".format(acc_log_test))
print("RF based on selected dataset")
print("FR CV Accuracy Score after selection:", scores.mean().round(3))
# print("Standard Deviation:", scores.std())

In [ ]:
lr=LogisticRegression(C=0.9, random_state=SEED, max_iter=100000)
# lr.fit(X_train,y_train)
scores = cross_val_score(lr, X_rfe, y, cv=cv_split, scoring = 'accuracy')
# acc_log_train = round(lr.score(X_train, y_train)*100,2) 
# acc_log_test = round(lr.score(X_test,y_test)*100,2)
print("LR based on selected dataset")
# print("Training Accuracy: % {}".format(acc_log_train))
# print("Testing Accuracy: % {}".format(acc_log_test))
print("LR CV Accuracy Score Mean:", scores.mean())
# print("Standard Deviation:", scores.std())

In [ ]:
lr.fit(X_rfe, y)
perm = PermutationImportance(lr, random_state=SEED).fit(X_rfe, y)
eli5.show_weights(perm, feature_names = X_rfe.columns.tolist())

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_rfe)
shap.summary_plot(shap_values[1], X_rfe, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X_rfe)

In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k='all')
fit = bestfeatures.fit(X_train, y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Feature','Score']  #naming the dataframe columns
print(featureScores.nlargest(15,'Score')) 

In [ ]:
features_kbest=featureScores.nlargest(15,'Score')['Feature'].tolist()
features_kbest

In [ ]:
lr=LogisticRegression(C=3, random_state=SEED)
scores = cross_val_score(lr, X[features_kbest], y, cv=cv_split, scoring = 'accuracy')
print("Kbest CV Acc Score Mean:", scores.mean().round(3))